In [1]:
#!pip install --upgrade beautifulsoup4

In [2]:
import requests
import pandas as pd
from datetime import datetime
from datetime import date
import time
import pickle
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

## 1. Build port 

In [3]:
response = requests.get(
    url='https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States'
)
print(response.status_code) # should be 200

200


## 2. Table Scrapping

In [4]:
response = requests.get(
      url='https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States'
#     url =r'https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States_(before_2000)'
)
print(response.status_code) # should be 200

soup = BeautifulSoup(response.text,'lxml')

# extract pure data 
data = []
colum_names = []
# find all sortabel tables
for i in soup.find_all(name='table'):
    
    # find all column names 
    for j in i.find_all(name='th'):
        colum_names.append(j.text.rstrip())
    
    # find each row
    for k in i.find_all(name='tr'):
        
        each_row ={}
        # each item in the row
        for index,item in enumerate(k.find_all(name='td')):
            each_row[colum_names[index]] = item.text.rstrip()
        
        data.append(each_row)
        
df = pd.DataFrame(data)

df = df.dropna(axis=0).reset_index(drop=True) 

df.head()

200


,Date,Location,Deaths,Injuries,Description
0,"February 29, 2000","Flint, Michigan",1,0,Shooting of Kayla Rolland: At Buell Elementary...
1,"May 26, 2000","Lake Worth, Florida",1,0,"13-year-old honor student, Nathaniel Brazill, ..."
2,"June 28, 2000","Seattle, Washington",2[n 1],0,58-year-old Director of the Division of Pathol...
3,"August 28, 2000","Fayetteville, Arkansas",2[n 1],0,"36-year-old James Easton Kelly, a PhD candidat..."
4,"September 26, 2000","New Orleans, Louisiana",0,2,"13 year-olds Darrel Johnson, and Alfred Anders..."


## 3. Scrapping Latitude and Longitude of citys

In [5]:
data = []
loc_href = []
for i in soup.find_all(name='table',attrs={'class':'sortable wikitable'}):
    for j in i.find_all(name='tr'):
        
        each_row ={}
        iter_ = iter(j.find_all('td'))
        while True:
            try:
                each_row['Date']=next(iter_).text.rstrip()
                loc = next(iter_)
                each_row['Location']=loc.text.rstrip()
                # find latitude and logitude
                re_loc = requests.get(
                    url='https://en.wikipedia.org{}'.format(loc.a['href'])
                )

                bs = BeautifulSoup(re_loc.text,'lxml')
                b = bs.find('span',attrs={'class':'plainlinks nourlexpansion'})
                each_row['cordinate'] = (b.find(name='a',attrs={"class":"external text", "rel":"nofollow"}).find(name='span',attrs={'class':'geo-dec'}).text)
                # end
                
                
                #loc_href.append(loc.a['href'])
                each_row['Deaths']=next(iter_).text.rstrip()
                each_row['Injuries']=next(iter_).text.rstrip()
                each_row['Description']=next(iter_).text.rstrip()
            except StopIteration: 
                break
            
        data.append(each_row)
        #print('\n')
        

KeyboardInterrupt: 

In [6]:
with open('raw_data.pkl','wb') as f:
    pickle.dump(data,f)

## 4. Data Clean 

In [7]:
df = pd.DataFrame(data[1:])

df = df.dropna(axis=0).reset_index(drop=True)

df[['City','State']]=df.Location.str.split(',',expand=True)
df.drop(columns=['Location'],inplace=True)

df['Deaths'] = df.Deaths.str.extract(r'(\d+)')
df['Injuries'] = df.Injuries.str.extract(r'(\d+)')


df[['Latitude','Longitude']] = df.cordinate.str.split(' ',expand=True)
df.drop(columns=['cordinate'],inplace =True)


df['Latitude'] = df.Latitude.str.extract(r'(\d+\.\d+)')
df['Longitude'] = df.Longitude.str.extract(r'(\d+\.\d+)')

df['Longitude'] = df['Longitude'].apply(lambda x: '-'+x)

df['Longitude']= df['Longitude'].astype(float)
df['Latitude'] = df['Latitude'].astype(float)

df['Date'] = df['Date'].apply(lambda x:datetime.strptime(x,'%B %d, %Y'))
df['Description'] =df['Description'].str.replace(r'(\[\d+\])*','')

In [ ]:
df.isna().any()

In [8]:
with open('processed_data.pkl','wb') as f:
    pickle.dump(df,f)

## 5.Data visualization

In [9]:
df = pd.read_csv('processed_data.csv')

In [10]:
df['Date'] = pd.to_datetime(df.Date)

In [11]:
df['Casualty'] =df['Deaths'].astype(int)+df['Injuries'].astype(int)
df['Year'] = df.Date.apply(lambda x:x.year)

In [12]:
df = df[df['Year']>=2010].reset_index(drop=True)

In [13]:
import altair as alt
from vega_datasets import data

airports = data.airports.url
states = alt.topo_feature(data.us_10m.url, feature='states')




# US states background
background = alt.Chart(states).mark_geoshape(
    fill='gray',
    stroke='white'
).properties(
    width=500,
    height=300
).project('albersUsa')



# airport positions on background
deaths = alt.Chart(df).transform_aggregate(
    latitude='mean(Latitude)',
    longitude='mean(Longitude)',
    Deaths='sum(Deaths)',
    Injuries = 'sum(Injuries)',
    groupby=['State','City']
).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.Size('Deaths:Q', title='Number of Cases',legend = None),
    color=alt.value('maroon'),
    #color= 'Deaths:Q',
    
    tooltip=['State:N','City:N','Deaths:Q','Injuries:Q']
).properties(
    title='Shooting casualties in US'
)

In [14]:
casualty = alt.Chart(df).mark_bar(opacity=0.7).transform_aggregate(
    Casualty = 'sum(Casualty)',
    Deaths='sum(Deaths)',
    Injuries = 'sum(Injuries)',
    groupby=['Year']
).encode(
    x='Year:O',
    y=alt.Y('Casualty:Q', stack=None),
    tooltip=['Year:O','Deaths:Q','Injuries:Q']
)
deaths_count = alt.Chart(df).mark_bar(opacity=0.7).transform_aggregate(
    Deaths='sum(Deaths)',
    groupby=['Year']
).encode(
    x='Year:O',
    y=alt.Y('Deaths:Q', stack=None),
    color = alt.value('maroon'),
    tooltip=['Year:O','Deaths:Q']
)
death_specific =alt.Chart(df[df['State'].isin([' Texas',' Oregon'])]).mark_bar(opacity=0.7).transform_aggregate(
    Deaths='sum(Deaths)',
    groupby=['Year']
).encode(
    x='Year:O',
    y=alt.Y('Deaths:Q', stack=None),
    color = alt.value('orange'),
    tooltip=['Year:O','Deaths:Q']
)

In [15]:
(background+deaths)|(casualty+deaths_count)

alt.HConcatChart(...)

## 6.Sort cases

In [17]:
df.groupby(['State','City']).City.agg(['count']).sort_values(by='count',ascending=False).head(15)

count
State           City                
 California     Los Angeles        7
 Texas          Houston            5
 Alabama        Mobile             4
 Tennessee      Nashville          4
                Memphis            4
 Pennsylvania   Philadelphia       4
 Michigan       Detroit            3
 Florida        Miami              3
 North Carolina Winston-Salem      3
 Illinois       Chicago            3
 Georgia        Savannah           3
                Augusta            3
                Albany             2
 Connecticut    New Haven          2
 Florida        Jacksonville       2

## 7.Get County information

In [25]:
county_table = pd.read_html("https://en.wikipedia.org/wiki/User:Michael_J/County_table", header=0)

In [30]:
df_county = county_table[0]

In [31]:
df_county.head()

,Sort [1],State,FIPS,County [2],County Seat(s) [3],Population(2010),Land Areakm²,Land Areami²,Water Areakm²,Water Areami²,Total Areakm²,Total Areami²,Latitude,Longitude
0,1,AL,1001,Autauga,Prattville,54571,1539.582,594.436,25.776,9.952,1565.358,604.388,+32.536382°,–86.644490°
1,2,AL,1003,Baldwin,Bay Minette,182265,4117.522,1589.784,1133.190,437.527,5250.712,2027.311,+30.659218°,–87.746067°
2,3,AL,1005,Barbour,Clayton,27457,2291.819,884.876,50.865,19.639,2342.684,904.515,+31.870670°,–85.405456°
3,4,AL,1007,Bibb,Centreville,22915,1612.481,622.582,9.289,3.587,1621.770,626.169,+33.015893°,–87.127148°
4,5,AL,1009,Blount,Oneonta,57322,1669.962,644.776,15.157,5.852,1685.119,650.628,+33.977448°,–86.567246°


## 8. Extract school name by RoBerta model and SpaCy

using QA model and NER techniques

In [18]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
model_name = "deepset/roberta-base-squad2"

# Load model
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
import spacy
from spacy import displacy

# Load english model: https://spacy.io/models
ent = spacy.load('en')

### Which school?

By asking "Which school?"
and looking for ORG tag.

In [19]:
res = []
for text in df['Description'].values:
    QA_input = {
    'question': 'Which school?',
    'context':text
    }
    res.append(nlp(QA_input))

In [42]:
lst_school = []
for i in res:
    lst_school.append(i['answer'])

In [44]:
#df.drop(columns=['Unnamed: 0'],inplace =True)

In [48]:
df["School"] = lst_school

In [72]:
# check text with ORG
def check_ner(text):
    doc = ent(text)
    count_org = 0
    count_error = 0
    text = []
    for i in doc.ents:
        if i.label_ =='ORG':
            text.append(i.text)
    combined_text = ' '.join(text)
    return combined_text
            

In [73]:
df["ner"] = df["Description"].map(lambda x:check_ner(x))

In [76]:
df.to_csv("with_school_data.csv")